# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [153]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [154]:

clean_weather = "~/desktop/clean_weather3.csv"

weather_data = pd.read_csv(clean_weather)

weather_data_df = pd.DataFrame(weather_data)

weather_data_df.head()

,Unnamed: 0,city,humidity,temp,lat,lng,clouds,wind
0,0,Thompson,65.0,299.15,55.74,55.74,75,2.6
1,1,Kapaa,74.0,301.07,22.08,22.08,90,5.7
2,2,Saint George,7.0,307.05,37.10,37.10,1,4.1
3,3,Jamestown,72.0,291.35,42.10,42.10,1,2.6
4,4,Mehrān,33.0,298.15,33.12,33.12,0,1.0


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [155]:
gmaps.configure(api_key=g_key)

In [156]:
locations = weather_data_df[["lat", "lng"]].astype(float)
humid = weather_data_df["humidity"].astype(float)

In [157]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humid, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [158]:


ideal_weather_df = weather_data_df.loc[(weather_data_df['temp'] > 290) & (weather_data_df['temp'] <305) & (weather_data_df['humidity']<40) & (weather_data_df['humidity']>20) & (weather_data_df['clouds']<30)]

#& (weather_data_df['temp'] <81) & (weather_data_df['humidity']<50) & (weather_data_df['humidity']>20)
#* A max temperature lower than 80 degrees but higher than 70.

 # * Wind speed less than 10 mph.

 # * Zero cloudiness.

 # * Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.
    
ideal_weather_df.head(50)

hotel_df = pd.DataFrame(ideal_weather_df)

hotel_df

,Unnamed: 0,city,humidity,temp,lat,lng,clouds,wind
4,4,Mehrān,33.0,298.15,33.12,33.12,0,1.00
66,69,Geraldton,37.0,294.15,-28.77,-28.77,1,9.80
74,77,Riverton,30.0,299.44,40.52,40.52,0,1.58
136,141,Atar,25.0,304.71,20.52,20.52,0,4.99
174,180,Sakakah,30.0,302.15,29.97,29.97,0,3.60
175,181,Tabas,22.0,304.15,33.60,33.60,2,4.10
237,244,Miles City,24.0,301.15,46.41,46.41,1,4.60
261,268,Emerald,37.0,294.20,-23.53,-23.53,10,6.20
286,293,Rūdbār,25.0,301.18,30.15,30.15,28,4.48
307,315,Ukiah,29.0,301.52,39.15,39.15,1,1.50


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [159]:


#hotels = []

hotel_df['Hotel Name'] = ''

hotel_df.head()

,Unnamed: 0,city,humidity,temp,lat,lng,clouds,wind,Hotel Name
4,4,Mehrān,33.0,298.15,33.12,33.12,0,1.00,
66,69,Geraldton,37.0,294.15,-28.77,-28.77,1,9.80,
74,77,Riverton,30.0,299.44,40.52,40.52,0,1.58,
136,141,Atar,25.0,304.71,20.52,20.52,0,4.99,
174,180,Sakakah,30.0,302.15,29.97,29.97,0,3.60,


In [181]:

#target_coordinates = 
#target_search = hotel
#target_radius = 5000
#target_type = lodging

params = {
    "keyword": "hotel" ,
    "radius": 5000,
    "types": "lodging",
    "key": g_key

}
for index, row in hotel_df.iterrows():
    
    lat = row['lat']
    lng = row['lng']
   
    
    params['location'] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    
    hotelname = requests.get(base_url, params=params)
    hotelname = hotelname.json()
    

    pprint(hotelname)
    
    
    
    
    

{'html_attributions': [], 'results': [], 'status': 'ZERO_RESULTS'}
{'html_attributions': [], 'results': [], 'status': 'ZERO_RESULTS'}
{'html_attributions': [], 'results': [], 'status': 'ZERO_RESULTS'}
{'html_attributions': [], 'results': [], 'status': 'ZERO_RESULTS'}
{'html_attributions': [], 'results': [], 'status': 'ZERO_RESULTS'}
{'html_attributions': [], 'results': [], 'status': 'ZERO_RESULTS'}
{'html_attributions': [], 'results': [], 'status': 'ZERO_RESULTS'}
{'html_attributions': [], 'results': [], 'status': 'ZERO_RESULTS'}
{'html_attributions': [], 'results': [], 'status': 'ZERO_RESULTS'}
{'html_attributions': [], 'results': [], 'status': 'ZERO_RESULTS'}
{'html_attributions': [], 'results': [], 'status': 'ZERO_RESULTS'}
{'html_attributions': [], 'results': [], 'status': 'ZERO_RESULTS'}
{'html_attributions': [], 'results': [], 'status': 'ZERO_RESULTS'}
{'html_attributions': [], 'results': [], 'status': 'ZERO_RESULTS'}
{'html_attributions': [], 'results': [], 'status': 'ZERO_RESUL

In [182]:
for index, row in hotel_df.iterrows():
    
    try:
        hotel_df.loc[index]['Hotel Name'] = hotelname['results'][0]['name']
    except:
        print("Missing")

    #NOTE: Something seems to be amiss with the way the dictionary/lists are being interpreted. As a result, 
    #my code doesn't work to add Hotel Name (using 'name') to the dataframe.

Missing
Missing
Missing
Missing
Missing
Missing
Missing
Missing
Missing
Missing
Missing
Missing
Missing
Missing
Missing
Missing
Missing
Missing
Missing
Missing
Missing
Missing
Missing


In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>city</dt><dd>{city}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["lat", "lng"]]

In [183]:
# Add marker layer ontop of heat map

hotel_df.head(80)



,Unnamed: 0,city,humidity,temp,lat,lng,clouds,wind,Hotel Name
4,4,Mehrān,33.0,298.15,33.12,33.12,0,1.00,
66,69,Geraldton,37.0,294.15,-28.77,-28.77,1,9.80,
74,77,Riverton,30.0,299.44,40.52,40.52,0,1.58,
136,141,Atar,25.0,304.71,20.52,20.52,0,4.99,
174,180,Sakakah,30.0,302.15,29.97,29.97,0,3.60,
175,181,Tabas,22.0,304.15,33.60,33.60,2,4.10,
237,244,Miles City,24.0,301.15,46.41,46.41,1,4.60,
261,268,Emerald,37.0,294.20,-23.53,-23.53,10,6.20,
286,293,Rūdbār,25.0,301.18,30.15,30.15,28,4.48,
307,315,Ukiah,29.0,301.52,39.15,39.15,1,1.50,


Missing


In [ ]:
marker_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=[f"Hotels: {name}" for name in hotel_df]
)


fig = gmaps.figure()
fig.add_layer(bank_layer)

fig
# Display figure